#데이터 불러오기

In [1]:
from google.colab import drive
drive.mount('/content/drive')

ModuleNotFoundError: No module named 'google.colab'

In [1]:
import pandas as pd
import os

#데이터 다 가져와 하나의 데이터프레임으로 묶기
csv_files = []
path = r"C:\Users\Hane\Downloads\졸업 프로젝트\알라딘 베스트셀러 Top 1000 1999~2024"
for root, dirs, files in os.walk(path):
    for file in files:
        if file.endswith('.csv'):
            csv_files.append(os.path.join(root, file))


df_original = pd.concat([pd.read_csv(os.path.join(path, file), encoding="utf-8-sig", dtype = {"ISBN" : 'str'}) for file in csv_files],ignore_index=False)


#데이터 탐색

In [9]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
Index: 112925 entries, 0 to 999
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   title               112925 non-null  object
 1   author              112773 non-null  object
 2   genre_id            112925 non-null  int64 
 3   genre               112545 non-null  object
 4   ISBN                112009 non-null  object
 5   description_aladin  102664 non-null  object
 6   description_naver   68110 non-null   object
 7   salesPoint          112925 non-null  int64 
 8   mallType            112925 non-null  object
dtypes: int64(2), object(7)
memory usage: 8.6+ MB


In [10]:
df_original.dtypes

title                 object
author                object
genre_id               int64
genre                 object
ISBN                  object
description_aladin    object
description_naver     object
salesPoint             int64
mallType              object
dtype: object

#전처리

In [11]:
#df_original = df_original.copy()
df_original = df_original.dropna(subset = ['ISBN']) #없어서는 안되는 키 값 결측치 제거
df_original = df_original.drop_duplicates(subset = 'ISBN', keep = 'first') #중복 제거

df_original = df_original.reset_index(drop=True) #인덱스 초기화


In [12]:
df_original.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48251 entries, 0 to 48250
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   title               48251 non-null  object
 1   author              48200 non-null  object
 2   genre_id            48251 non-null  int64 
 3   genre               48107 non-null  object
 4   ISBN                48251 non-null  object
 5   description_aladin  41955 non-null  object
 6   description_naver   29000 non-null  object
 7   salesPoint          48251 non-null  int64 
 8   mallType            48251 non-null  object
dtypes: int64(2), object(7)
memory usage: 3.3+ MB


#상세설명 토픽 추출 - LDA 모델

상세설명 분석용 추가 전처리

In [13]:
df = df_original.copy()

#네이버 알라딘 상세설명 둘 다 없는거 드랍
df = df[~(df['description_aladin'].isnull() & df['description_naver'].isnull())]

#네이버 상세설명 특수문자 등 전처리
df['description_naver'] = df['description_naver'].str.replace(pat=r'[^\w\s가-힣]', repl=r' ', regex=True)
df['description_naver'] = df['description_naver'].str.replace(pat=r'[\n\d]', repl=r' ', regex=True)
#알라딘 상세설명 전처리
df['description_aladin'] = df['description_aladin'].str.replace(pat=r'[^\w\s가-힣]', repl=r' ', regex=True)
df['description_aladin'] = df['description_aladin'].str.replace(pat = r'[&lt&gt]', repl=r'', regex=True)     #&lt &gt = < >

#네이버 비었으면 알라딘 넣기
df['description_naver'] = df['description_naver'].fillna(df['description_aladin'])

한국어 불용어 제거 : BoW는 각 단어별 빈도를 저장하는거라 불용어 제거해야 단어들끼리 모임

In [14]:
import konlpy
from konlpy.tag import Okt
import tqdm

In [15]:
#형태소 분석 후 명사 중 불용어에 포함되지 않은 것만 저장
Okt = konlpy.tag.Okt()
#komoran = konlpy.tag.Komoran()

#불용어 사전
stopword_data_path = r'C:\Users\Hane\Downloads\졸업 프로젝트/stopwords-ko.txt'
#불용어 제거 전 dictionary length : 100000 (사실상 초과)
#불용어 제거 후 dictionary length : 34062
with open(stopword_data_path, 'r', encoding='utf-8') as f:
  stopwords = [line.strip() for line in f.readlines()]

pd.options.mode.chained_assignment = None  # default='warn'
def process_text(text):
  Okt_morphs = Okt.pos(text)
  return ' '.join([word for word, pos in Okt_morphs if pos == 'Noun' and word not in stopwords and len(word)>1])

df['description_naver'] = df['description_naver'].apply(lambda x : process_text(x))


for i in range(len(df['description_naver'])):
  Okt_morphs = Okt.pos(df['description_naver'].iloc[i])

  temp = []
  for word, pos in Okt_morphs:
    if pos == 'Noun' and word not in stopwords and len(word)>1: #명사, 불용어 비포함, 2글자 이상
        temp.append(word)

  df['description_naver'].iloc[i] = ' '.join(temp)
  if i % 10000 == 0:
    print(i)

In [16]:
df['description_naver'].loc[15441:15450]
df['description_naver'].isnull().sum()

0

In [17]:
#original data 전처리 체크포인트 저장
#df.to_csv(r"C:\Users\Hane\Downloads\졸업 프로젝트\LDA 결과들\df_preprocessed_checkpoint.csv", encoding = 'utf-8-sig', index=False)

LDA 모델 생성

sklearn vs gensim LDA : https://medium.com/@benzgreer/sklearn-lda-vs-gensim-lda-691a9f2e9ab7
https://www.kaggle.com/code/morrisb/compare-lda-topic-modeling-in-sklearn-and-gensim

In [2]:
df = pd.read_csv(r"C:\Users\Hane\Downloads\졸업 프로젝트\LDA 결과들\df_preprocessed_checkpoint.csv", encoding = 'utf-8-sig', dtype = {'description_naver' : 'string'})
df = df.dropna(subset = ['description_naver'])
#df[df['description_naver'].isnull()]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 44409 entries, 0 to 44458
Data columns (total 9 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   title               44409 non-null  object
 1   author              44388 non-null  object
 2   genre_id            44409 non-null  int64 
 3   genre               44386 non-null  object
 4   ISBN                44409 non-null  object
 5   description_aladin  41936 non-null  object
 6   description_naver   44409 non-null  string
 7   salesPoint          44409 non-null  int64 
 8   mallType            44409 non-null  object
dtypes: int64(2), object(6), string(1)
memory usage: 3.4+ MB


In [3]:
import gensim
from gensim import corpora

doc = df['description_naver'].apply(lambda x : x.split()).tolist()#토큰화

#Bag of words 만들기
#no_below개의 문서 아래로 나오는 단어, no_above 비율보다 자주나오는 단어 제거.
#이걸 하니 모든 토픽에 공통적으로 들어가있던 '작품'같은 의미없는 단어 사라짐.
#for i in range(0,11):
#    dictionary = corpora.Dictionary(doc)
#    dictionary.filter_extremes(no_below=i, no_above=0.5)
#    print(f"no_below : {i} -> len : {len(dictionary)}")


In [4]:
from gensim import corpora

#no_below 3으로 지정
dictionary = corpora.Dictionary(doc)
dictionary.filter_extremes(no_below=3, no_above=0.5)
corpus = [dictionary.doc2bow(text) for text in doc]

lda 모델 만드는 동시에 최적의 토픽 개수 찾기 
https://m.blog.naver.com/j7youngh/222929522859
https://towardsdatascience.com/practical-guide-to-topic-modeling-with-lda-05cd6b027bdf

In [26]:
import gensim
from gensim.models import CoherenceModel

NUM_TOPICS = 100

def go(CHUNKSIZE, PASSES, ITERATIONS, EVAL_EVERY, UPDATE_EVERY):
    #gensim LDA는 한 번 들어온 Chunk가 문서의 전체 길이인 것으로 파악하게 된다 : chunk는 최대한 많이.
    #CHUNKSIZE = 10000 #10000     #한번에 트레이닝 처리할 corpus 문서의 개수
    #PASSES = 3         #첫 chunk부터 마지막 chunk까지 학습 반복 횟수. 
    #ITERATIONS = 10      #문서 당 반복 횟수
    #EVAL_EVERY = 1    #eval_every번째 chunk마다 perplexity 평가
    #UPDATE_EVERY = 1
    
    f = open(f"chunk={CHUNKSIZE}, pass={PASSES}, iter={ITERATIONS}, eval={EVAL_EVERY}, update={UPDATE_EVERY}.txt", "w")
    
    coherence_score = []
    perplexity_score = []
    
    for i in range(50,151,10):
        ldamodel = gensim.models.ldamodel.LdaModel(corpus,
                                                   id2word = dictionary,
                                                   num_topics = i,
                                                   passes = PASSES,
                                                   chunksize = CHUNKSIZE,
                                                   iterations = ITERATIONS,
                                                   eval_every = EVAL_EVERY,
                                                   update_every = UPDATE_EVERY,
                                                   alpha = 'auto', #문서 내 토픽분포.낮으면 적은 수의 토픽에, 높으면 많은 수의 토픽에 집중
                                                   eta = 'auto', #토픽 내 단어분포.낮으면 적은 수의 단어에, 높으면 많은 수의 단어가 포함.                                              
                                                  )
        coherence_model = CoherenceModel(model = ldamodel, texts = doc, dictionary = dictionary, coherence = 'c_v')
        coherence_lda = coherence_model.get_coherence()
        perplexity_lda = ldamodel.log_perplexity(corpus)
        print(i, "Coherence Score : ", coherence_lda)
        print(i, "Perplexity Score : ", perplexity_lda)
        f.write(f"{i}, Coherence Score : {coherence_lda}, Perplexity Score : {perplexity_lda}\n")
        
        coherence_score.append(coherence_lda)
        perplexity_score.append(perplexity_lda)
    
    f.close()

In [ ]:
#최적 토픽 개수 연구기기
go(20000, 3, 10, 1, 1)
go(30000, 3, 10, 1, 1)
#go(40000, 3, 10, 1, 1)

#go(10000, 5, 10, 1, 1)
#go(10000, 10, 10, 1, 1)
#go(10000, 15, 10, 1, 1)

50 Coherence Score :  0.45442405832256894
50 Perplexity Score :  -8.856714549873665
60 Coherence Score :  0.4459678567522543
60 Perplexity Score :  -9.04122523364301
70 Coherence Score :  0.4559235743395801
70 Perplexity Score :  -9.17985948670066
80 Coherence Score :  0.46284633100239175
80 Perplexity Score :  -9.3337645846174
90 Coherence Score :  0.47412446858888374
90 Perplexity Score :  -9.512588696718018
100 Coherence Score :  0.457381043483188
100 Perplexity Score :  -9.659433527981884
110 Coherence Score :  0.44080207733930576
110 Perplexity Score :  -9.81801304927116
120 Coherence Score :  0.465536983289854
120 Perplexity Score :  -9.971667918154626


In [14]:
import time
import gensim
from gensim.models import CoherenceModel

#gensim LDA는 한 번 들어온 Chunk가 문서의 전체 길이인 것으로 파악하게 된다 : chunk는 최대한 많이.
CHUNKSIZE = 20000  #한번에 트레이닝 처리할 corpus 문서의 개수
PASSES = 10         #첫 chunk부터 마지막 chunk까지 학습 반복 횟수. 
ITERATIONS = 30    #문서 당 반복 횟수
EVAL_EVERY = 1     #eval_every번째 chunk마다 perplexity 평가
UPDATE_EVERY = 1
        
coherence_score = []
perplexity_score = []
    
start_time = time.time()
ldamodel = gensim.models.ldamodel.LdaModel(corpus,
                                           id2word = dictionary,
                                           num_topics = 100,
                                           passes = PASSES,
                                           chunksize = CHUNKSIZE,
                                           iterations = ITERATIONS,
                                           eval_every = EVAL_EVERY,
                                           update_every = UPDATE_EVERY,
                                           alpha = 'auto', #문서 내 토픽분포.낮으면 적은 수의 토픽에, 높으면 많은 수의 토픽에 집중
                                           eta = 'auto', #토픽 내 단어분포.낮으면 적은 수의 단어에, 높으면 많은 수의 단어가 포함.                                              
                                          )
coherence_model = CoherenceModel(model = ldamodel, texts = doc, dictionary = dictionary, coherence = 'c_v')
coherence_lda = coherence_model.get_coherence()
perplexity_lda = ldamodel.log_perplexity(corpus)
print("Coherence Score : ", coherence_lda)
print("Perplexity Score : ", perplexity_lda)

end_time = time.time()
print(f"전체 실행 시간 : {end_time - start_time:.2f}초")

Coherence Score :  0.48465390303188327
Perplexity Score :  -9.343666175157164
전체 실행 시간 : 3183.48초


In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(2, 1, figsize = (8,6))
x = range(10, 61)
y1 = coherence_score
y2 = perplexity_score

ax[0].plot(x, y1, label = 'Coherence Score')
ax[0].set_title('Coherence Score')
ax[0].set_xlabel('Num Topics (k)')
ax[0].set_ylabel('Score')

ax[1].plot(x, y2, label = 'Perplexity Score')
ax[1].set_title('Perplexity Score')
ax[1].set_xlabel('Num Topics (k)')
ax[1].set_ylabel('Score')

plt.show()

토픽별 (기여도 * 단어) 출력

In [15]:
topics = ldamodel.print_topics(num_words=6)
for topic in topics:
    print(topic)

(69, '0.113*"놀이" + 0.038*"인형" + 0.028*"다윈" + 0.027*"야마" + 0.020*"벌레" + 0.018*"거울"')
(90, '0.061*"면접" + 0.035*"검사" + 0.032*"약점" + 0.030*"카드" + 0.024*"도로" + 0.022*"병원"')
(5, '0.032*"인터넷" + 0.031*"보험" + 0.026*"통증" + 0.023*"문제" + 0.020*"법인" + 0.019*"세금"')
(11, '0.025*"코딩" + 0.021*"장판" + 0.020*"명언" + 0.018*"폭탄" + 0.017*"스케치" + 0.017*"편의점"')
(27, '0.135*"수학" + 0.034*"블로그" + 0.015*"개념" + 0.013*"논문" + 0.013*"도안" + 0.013*"방법"')
(99, '0.070*"공인" + 0.054*"서점" + 0.045*"중개사" + 0.026*"기록" + 0.024*"연기" + 0.019*"합격"')
(79, '0.079*"의사" + 0.044*"환자" + 0.036*"치료" + 0.029*"정신과" + 0.026*"연구소" + 0.021*"오공"')
(63, '0.075*"요리" + 0.057*"레시피" + 0.054*"음식" + 0.035*"재료" + 0.023*"메뉴" + 0.020*"가지"')
(70, '0.033*"회복" + 0.028*"피아노" + 0.023*"할멈" + 0.021*"총론" + 0.019*"복지" + 0.019*"수록"')
(80, '0.065*"장애" + 0.024*"타임" + 0.017*"단독" + 0.016*"젠더" + 0.015*"정상" + 0.015*"자폐"')
(13, '0.072*"여행" + 0.042*"기억" + 0.023*"이야기" + 0.023*"작가" + 0.014*"도시" + 0.013*"풍경"')
(16, '0.033*"한국" + 0.024*"정치" + 0.024*"사회" + 0.023*"전쟁" + 0.020*"미국

문서별 (토픽, 기여도) 출력
*주의* 문서가 60%A이고 40%B이면 'A로분류되지만 모델이 애매하게 분류한다'가 아니라 'A에도 분류되고 확실하게 B로도 분류된다'로 해석해야함.

In [16]:
import numpy as np

#출력해보기
for i, topic_list in enumerate(ldamodel[corpus]):
    if i==5:
        break
    dominant_topic = max(topic_list, key=lambda x: x[1])
    print(f"{i} 번째 문서의 가장 중요한 토픽은 {dominant_topic[0]} 번, 비율은 {dominant_topic[1]:.4f}")
    print(i,'번째 문서의 topic 비율은',topic_list)



0 번째 문서의 가장 중요한 토픽은 38 번, 비율은 0.3861
0 번째 문서의 topic 비율은 [(0, 0.13323793), (3, 0.041618586), (38, 0.38614646), (90, 0.04355418), (92, 0.33185232)]
1 번째 문서의 가장 중요한 토픽은 34 번, 비율은 0.6757
1 번째 문서의 topic 비율은 [(5, 0.069573104), (34, 0.67569184), (41, 0.11098994), (94, 0.06715095)]
2 번째 문서의 가장 중요한 토픽은 20 번, 비율은 0.4215
2 번째 문서의 topic 비율은 [(20, 0.4214666), (51, 0.13823569), (59, 0.29080552), (68, 0.06478127)]
3 번째 문서의 가장 중요한 토픽은 65 번, 비율은 0.2873
3 번째 문서의 topic 비율은 [(5, 0.049910456), (34, 0.15946518), (38, 0.08843921), (52, 0.038338825), (55, 0.0441782), (65, 0.28730607), (86, 0.13297328), (89, 0.1047164), (94, 0.04150257)]
4 번째 문서의 가장 중요한 토픽은 93 번, 비율은 0.3160
4 번째 문서의 topic 비율은 [(15, 0.046039358), (18, 0.13385358), (23, 0.3134631), (87, 0.042903382), (89, 0.10499223), (93, 0.31600937)]


In [ ]:
#저장하기
lda_results = [None] * len(df)

for i, topic_list in enumerate(ldamodel[corpus]):
    dominant_topic = max(topic_list, key=lambda x: x[1])
    lda_results[i] = dominant_topic[0]


In [21]:
df['lda_result'] = lda_results

In [22]:
df.head(3)

,title,author,genre_id,genre,ISBN,description_aladin,description_naver,salesPoint,mallType,lda_result
0,국어사개설 - 신정판,이기문 (지은이),8292,국내도서>대학교재/전문서적>어문학계열>국어국문학>국어사/방언론,9788976263728,국어의 계통과 형성 문자체계를 먼저 살피고 이어 고대국어 전기 중세국어와 후기 ...,국어 계통 형성 문자체계 먼저 살피 고대 국어 전기 중세 국어 후기 중세 국어 근대...,5758,BOOK,38
1,몰입의 즐거움 - 개정판,"미하이 칙센트미하이 (지은이), 이희재 (옮긴이)",51378,국내도서>인문학>교양 인문학,9788973378869,수천 명의 개인을 대상으로 한 과학적 조사 결과에 근거한 이 책은 하루하루 살아가면...,수천 명의 개인 대상 과학 조사 결과 근거 하루하루 맞닥뜨리 사건 일상 생활 초점 ...,10395,BOOK,34
2,플리머스에서의 즐거운 건맨생활 - 1999 제23회 오늘의 작가상 수상작,우광훈 (지은이),50994,국내도서>소설/시/희곡>한국소설>2000년대 이전 한국소설,9788937403224,1999년 제23회 오늘의 작가상 수상작 시간적 배경은 서기 2237년 공간적 ...,오늘 작가 수상작 배경 서기 공간 배경 가상 체험 프로그램 통해 세기 미국 서부 플...,46,BOOK,20


In [26]:
df[df['lda_result'].isnull()]

,title,author,genre_id,genre,ISBN,description_aladin,description_naver,salesPoint,mallType,lda_result


In [24]:
pip install pyLDAvis

Note: you may need to restart the kernel to use updated packages.Collecting pyLDAvis
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   -- ------------------------------------- 0.2/2.6 MB ? eta -:--:--
   -- ------------------------------------- 0.2/2.6 MB ? eta -:--:--
   ------ --------------------------------- 0.4/2.6 MB 3.3 MB/s eta 0:00:01
   ---------- ----------------------------- 0.7/2.6 MB 4.0 MB/s eta 0:00:01
   -------------- ------------------------- 0.9/2.6 MB 5.0 MB/s eta 0:00:01
   -------------- ------------------------- 0.9/2.6 MB 5.0 MB/s eta 0:00:01
   -------------------- ------------------- 1.4/2.6 MB 4.1 MB/s eta 0:00:01
   ---------------------------- ----------- 1.8/2.6 MB 5.3 MB/s eta 0:00:01
   ---------------------------- ----------- 1.8/2.6 MB 5.3 MB/s eta 0:00:01
   ------------------------------- -------- 2.1/2.6 MB 4.2 MB/s eta 0:00:01
   ---------------------------------- ----- 2.2/2.6 MB 4.3 MB/s eta 0:00:01
   ---------------------


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [25]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

In [ ]:
모델 저장, 체크포인트 저장

In [27]:
ldamodel.save('lda_model.model')
#ldamodel.load('lda_model.model')

dictionary.save('dictionary.dict')
#from gensim.corpora import Dictionary
#dictionary = Dictionary.load('dictionary.dict')

from gensim.corpora import MmCorpus
MmCorpus.serialize('corpus.mm', corpus)
#corpus = MmCorpus('corpus.mm')

df.to_csv(r"C:\Users\Hane\Downloads\졸업 프로젝트\LDA 결과들\df_lda_checkpoint.csv", encoding = 'utf-8-sig', index=False)

#장르 핸들링

데이터 불러오기

In [ ]:
df = df_original.copy()
cid_data = pd.read_excel("/content/drive/MyDrive/알라딘 베스트셀러 Top 1000 1999~2024/aladin_Category_CID_20210927.xls")

In [ ]:
#열 재정의, 필요없는 행 삭제
cid_data.columns = cid_data.iloc[1]
cid_data = cid_data.drop(index = [0,1])
cid_data.head(3)

1,CID,카테고리명,몰,1Depth,2Depth,3Depth,4Depth,5Depth
2,1230,가정/요리/뷰티,국내도서,가정/요리/뷰티,NaN,NaN,NaN,NaN
3,90452,가계부,국내도서,가정/요리/뷰티,가계부,NaN,NaN,NaN
4,53471,건강요리,국내도서,가정/요리/뷰티,건강요리,NaN,NaN,NaN


groupby 기준 찾기

In [ ]:
grouped_cid1 = cid_data.groupby(['1Depth']).agg({'CID': lambda x: list(x)}).reset_index()

print("[1Depth] groupby 전체 length : ", len(grouped_cid1))
print("CID가 하나만 속하는 데이터 :", len(grouped_cid1[grouped_cid1['CID'].apply(len) == 1]))
grouped_cid1[100:105]

[1Depth] groupby 전체 length :  111
CID가 하나만 속하는 데이터 : 11


1,1Depth,CID
100,초등학교참고서,"[50246, 52102, 52147, 52151, 52149, 52148, 521..."
101,취미/스포츠,[15837]
102,컴퓨터,"[90859, 97884, 97885, 97886, 97887, 97888, 978..."
103,컴퓨터/모바일,"[351, 2630, 2643, 2632, 2672, 2669, 7439, 2633..."
104,클래식,"[5915, 15017, 127045, 5933, 110089, 110095, 11..."


In [ ]:
grouped_cid2 = cid_data.groupby(['1Depth', '2Depth']).agg({'CID': lambda x: list(x)}).reset_index()

print("[1Depth, 2Depth] groupby 전체 length : ", len(grouped_cid2))
print("CID가 하나만 속하는 데이터 :", len(grouped_cid2[grouped_cid2['CID'].apply(len) == 1]))
grouped_cid2[100:105]

[1Depth, 2Depth] groupby 전체 length :  1854
CID가 하나만 속하는 데이터 : 996


1,1Depth,2Depth,CID
100,가정/요리/뷰티,전문가/연예인/블로거 요리,"[53475, 56819]"
101,가정/요리/뷰티,제과제빵,"[53481, 56825]"
102,가정/요리/뷰티,집/인테리어,"[53486, 90456, 90455]"
103,가정/요리/뷰티,패션/뷰티,"[53488, 53507, 53504, 53506, 53508, 53505, 568..."
104,가정/원예/인테리어,가정,"[97189, 103031, 103032, 103033, 103034, 103035..."


In [ ]:
grouped_cid3 = cid_data.groupby(['1Depth', '2Depth','3Depth']).agg({'CID': lambda x: list(x)}).reset_index()

print("[1Depth, 2Depth, 3Depth] groupby 전체 length : ", len(grouped_cid3))
print("CID가 하나만 속하는 데이터 :", len(grouped_cid3[grouped_cid3['CID'].apply(len) == 1]))
grouped_cid3[100:105]

[1Depth, 2Depth, 3Depth] groupby 전체 length :  6837
CID가 하나만 속하는 데이터 : 4509


1,1Depth,2Depth,3Depth,CID
100,Gift,어린이에게,초등 전학년,"[6087, 40707]"
101,Gift,어린이에게,한글을 배워요!,"[6092, 40712]"
102,Gift,특별한 날,감사/축하,"[6078, 40699]"
103,Gift,특별한 날,결혼,"[6075, 40696]"
104,Gift,특별한 날,새집,"[6152, 40723]"


**grouped2** 기준 원본데이터에 추가

In [ ]:
#빠른 매핑을 위한 딕셔너리 만들기
depth2_dict = {}
for i, items in enumerate(grouped_cid2['CID']):
  for id in list:
    depth2_dict[id] = grouped_cid2['2Depth'].index[i]


In [ ]:
#적용
df['genre_code'] = df['genre_id'].map(depth2_dict).astype('Int32')
df['genre_code'] = df['genre_code'].fillna(0).astype('Int32')
df.head(3)

,title,author,genre_id,genre,ISBN,description_aladin,description_naver,salesPoint,mallType,genre_code
0,내 어머니 이야기 세트 - 전4권,김은성 (지은이),3748,국내도서>만화>논픽션/휴먼만화,9788959199990,소설가 김영하의 강력 추천을 받으며 화제가 된 만화 &lt;내 어머니 이야기&gt;...,한 사람 한 사람의 존재와 삶이 모여 역사가 된다!\n\n2014년 완간되었다가 ...,9760,BOOK,468
1,고요할수록 밝아지는 것들 - 혜민 스님과 함께 지혜와 평온으로 가는 길,혜민 (지은이),51371,국내도서>에세이>한국에세이,9791187498384,"""가끔은 고요함 속에서 나를 만나는 시간을 선물하세요."" &lt;멈추면, 비로소 보...",▶ [100쇄 기념 에디션]은 인터넷/모바일 주문 전용입니다.\n▶ 영업점과 바로드...,16915,BOOK,927
2,해커스 토익 기출 VOCA (보카) - 주제별 연상암기로 토익 영단어 30일 완성!...,David Cho (지은이),49857,국내도서>외국어>토익>Vocabulary,9788965422785,토익 기초 단어부터 고난도 어휘까지 모든 난이도의 단어를 수록했다. 토익 빈출?주제...,주제별 연상암기로 토익 영단어 30일 완성\n\n1. 토익 기출 보카 30일 완성!...,217557,BOOK,1120


In [ ]:
print("결측값 개수 : ", df['genre_code'].isnull().sum())

cid_data[cid_data['CID'] == 107436]

결측값 개수 :  0


1,CID,카테고리명,몰,1Depth,2Depth,3Depth,4Depth,5Depth
